## Code preamble Import packages

In [241]:
import pandas as pd
import numpy as np
import json
import requests
import altair as alt
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

## Functions

### Writing figure tex

In [242]:
def format_strings(test_str):
    test_str = test_str.replace("_","")
    for ele in test_str:
        if ele.isdigit():
            test_str = test_str.replace(ele,  num_to_word(ele))
    return test_str
def num_to_word(num):
    n_dict = {0:'zero', 1:'one', 2:'two', 3:'three', 4:'four', 5:'five', 6:'six', 
              7:'seven',8:'eight', 9:'nine', 10:'ten', 11:'eleven', 12:'twelve',
              13:'thirteen', 14:'fourteen', 15: 'fifteen'}
    return n_dict[int(num)].title()


In [243]:
def pathway_figure_tex(label, fname, score, filter):
    list_str= [label, fname]
    method, fname = [format_strings(i_str) for i_str in list_str]
    score_lab = score.title().replace(" ", "")
    filter_lab = filter.title().replace(" ", "")
    beg_str = "\n \\newcommand{\\figurePathway"+label+score_lab+filter_lab+"}{\\begin{subfigure}[t]{\doublefigurewidth} \n"
    mid_str = "\includegraphics[width=\linewidth] {"+fname+"} \n"
    cap_str = "\caption{Pathway enrichment for the clustering method: "+label+". \n Showing the "+filter+" based on the "+score+".\label{fig::"+label+score_lab+filter_lab+"}}"
    end_str = "\end{subfigure} \n} \n"
    return(beg_str+mid_str+cap_str+end_str)

### Set the directories

In [244]:
path_dir_clone = "../../PheWAS-cluster-main/working-example/data"
path_dir = "../NealesLabData"
tex_dir = "../../../Documents/Reports/Clustering"
path_plots = "./Darrous_plots_all/"

### Load the clusters

In [245]:
clustering_info = pd.read_csv(path_dir+"/cluster_labels.csv", index_col=0)
clustering_info.head()
cluster_method_labels = list(clustering_info.index)
cluster_method_labels.remove("SpectralProposed")
cluster_method_labels.remove("LloydRProposed")


In [246]:
clust_df = pd.read_csv(path_dir+"/ours_clusters.csv", index_col = 0)
clust_df.head()

,body_size,1697,1717,20015_irnt,2306,23101_irnt,23102_irnt,23105_irnt,23106_irnt,23107_irnt,...,KmeansCosineMinibatchSixProposed,KmedoidsCosineSixProposed,GMMZeroProposed,GMMOneProposed,BirchZeroProposed,BirchOneProposed,BirchTwoProposed,SpectralProposed,KmeansCosineTwoMinAICProposed,KmedoidsCosineThreeMinAICProposed
rs115866895,-1.384558,0.078427,0.064179,1.313715,-0.390765,-1.428121,-1.395080,-1.765438,2.115427,1.865250,...,1,3,1,4,227,0,0,NaN,1,1
rs4648450,-1.280939,0.667544,0.025040,0.907814,0.041965,-2.116211,-2.090042,-2.321914,2.290245,1.904075,...,1,3,1,1,252,0,0,NaN,1,1
rs12024554,-1.758457,1.511510,0.407248,4.120851,0.602471,-0.676939,-0.701834,-0.828185,2.797708,2.548867,...,4,3,0,0,253,0,0,NaN,1,1
rs1097327,-1.027856,-0.284528,-0.186588,1.438508,-0.030276,-1.195410,-1.167861,-1.501871,1.218832,1.538645,...,4,3,0,0,300,0,0,NaN,1,1
rs3737992,-0.226169,-1.735855,0.039532,-1.505157,-0.230216,-2.347974,-2.330429,-2.498627,1.603422,1.071926,...,1,1,1,4,195,0,0,NaN,1,1


### Load the PCAs

In [247]:
pc_ours_df = pd.read_csv(path_dir+"/pcas_ours.csv", index_col=0)
pc_ours_with_exp_df = pd.read_csv(path_dir+"/pcas_ours_with_exp.csv", index_col=0)
pc_pap_df = pd.read_csv(path_dir+"/pcas_paper.csv", index_col=0)

### Load the Genes-Snps
The `Genes` have been found using the function `SNP2GENE` from [FUMA](https://fuma.ctglab.nl/). This requires the `GWAS` data to be uploaded with the following compulsary columns:
* Either Chromosome and Position or rsid.
* Effect Allele
* Sample size.
This must be input as a txt file with space separated data and `\n` separated rows.

In [248]:
genes_df = pd.read_csv(path_dir+'/genes.txt', sep="\t", header=0)
genes_df.head()

,ensg,symbol,chr,start,end,strand,type,entrezID,HUGO,pLI,ncRVIS,posMapSNPs,posMapMaxCADD,minGwasP,IndSigSNPs,GenomicLocus
0,ENSG00000197530,MIB2,1,1550795,1565990,1,protein_coding,142678.0,MIB2,0.000159,0.030973,4,3.164,NaN,rs115866895,1
1,ENSG00000189409,MMP23B,1,1567474,1570639,1,protein_coding,8510.0,MMP23B,NaN,NaN,5,3.164,NaN,rs115866895,1
2,ENSG00000248333,CDK11B,1,1570603,1590473,-1,protein_coding,728642.0,CDK11B,0.728294,NaN,34,11.560,1.617240e-15,rs115866895,1
3,ENSG00000189339,SLC35E2B,1,1592939,1624167,-1,protein_coding,728661.0,SLC35E2B,0.621002,-0.794611,34,11.560,1.617240e-15,rs115866895,1
4,ENSG00000157873,TNFRSF14,1,2487078,2496821,1,protein_coding,8764.0,TNFRSF14,0.819999,-0.131268,1,1.637,NaN,rs4648450,2


In [249]:
gene_snps = genes_df[["symbol", "IndSigSNPs"]]
gene_snps.value_counts()
full_gene_list = np.unique(gene_snps["symbol"].values)

### Construct the Clusters in terms of Genes

In [250]:
snp_clusters_df = clust_df[clust_df.columns[clust_df.columns.isin(cluster_method_labels)]]
cluster_method_labels = [lab for lab in cluster_method_labels if lab in clust_df.columns]
gene_clust_df = pd.DataFrame(columns = cluster_method_labels)
snp_clusters_df

,KmeansLloydStdFourProposed,KmeansLloydStdFourDarrous,KmeansLloydStdSixProposed,KmeansLloydStdSixDarrous,KmeansElkanStdFourProposed,KmeansElkanStdFourDarrous,KmedoidsEuclideanProposedFour,KmeansCosineSixProposed,DBScanZeroProposed,DBScanOneProposed,DBScanTwoProposed,KmeansCosineMinibatchSixProposed,KmedoidsCosineSixProposed,GMMZeroProposed,GMMOneProposed,BirchZeroProposed,BirchOneProposed,BirchTwoProposed,KmeansCosineTwoMinAICProposed,KmedoidsCosineThreeMinAICProposed
rs115866895,1,1,2,4,1,1,0,1,0,-1,-1,1,3,1,4,227,0,0,1,1
rs4648450,3,2,1,0,3,2,0,1,-1,-1,-1,1,3,1,1,252,0,0,1,1
rs12024554,1,0,2,4,1,0,0,2,-1,-1,-1,4,3,0,0,253,0,0,1,1
rs1097327,3,3,4,3,3,3,0,3,-1,-1,-1,4,3,0,0,300,0,0,1,1
rs3737992,1,2,1,0,1,2,3,1,0,-1,-1,1,1,1,4,195,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
rs8134638,0,0,0,4,0,0,2,4,-1,-1,-1,2,5,2,2,43,1,1,0,2
rs2837398,0,0,0,4,0,0,1,4,-1,-1,-1,2,4,2,2,160,1,1,0,2
rs76040172,3,1,4,5,3,1,0,1,0,-1,-1,1,3,1,4,251,0,0,1,1
rs6001870,3,2,1,0,3,2,3,3,0,-1,-1,4,1,0,3,323,0,0,1,1


In [251]:
gene_clust_dict = {} #pd.DataFrame(index = genes_df.symbol.values)
for clust_lab in cluster_method_labels:
    # Get the clusters by snps for cluster method clust_lab
    clust_single = snp_clusters_df.loc[:, clust_lab]
    # Get the possible cluster numbers
    clust_nums = clust_single.unique()
    gene_clust_single_df = pd.DataFrame(
        columns = [clust_lab+"_"+str(c_n) for c_n in clust_nums]
    )
    for c_num in clust_nums:
        col_lab = clust_lab+"_"+str(c_num)
        # snps matching clusters
        clust_snps = list(clust_df[clust_single == c_num].index)
        # genes prevent in those snps
        genes = list(genes_df[genes_df["IndSigSNPs"].isin(clust_snps)].symbol.unique())
        # genes marked with 1 if in cluster 0 if not.
        gene_clust_c_num_df = pd.DataFrame(
            index = genes,
            data = {
                col_lab: 1
            }
        )
        extra_genes = list(np.setdiff1d(full_gene_list, genes))
        gene_clust_not_in_df = pd.DataFrame(
            index = extra_genes,
            data = {
                col_lab: 0
            }
        )
        gene_clust_c_num_df = pd.concat([gene_clust_c_num_df, gene_clust_not_in_df],
                                        axis = 0
        )
        gene_clust_single_df[col_lab] = gene_clust_c_num_df
    gene_clust_dict[clust_lab] = gene_clust_single_df


In [252]:
for lab in gene_clust_dict:
    gene_clust_dict[lab].to_csv(path_dir+"/gene_clusters_" + lab + ".csv",
                     header = True,
                     index= True,
                     sep = ",")
genes_df

,ensg,symbol,chr,start,end,strand,type,entrezID,HUGO,pLI,ncRVIS,posMapSNPs,posMapMaxCADD,minGwasP,IndSigSNPs,GenomicLocus
0,ENSG00000197530,MIB2,1,1550795,1565990,1,protein_coding,142678.0,MIB2,1.585433e-04,0.030973,4,3.164,NaN,rs115866895,1
1,ENSG00000189409,MMP23B,1,1567474,1570639,1,protein_coding,8510.0,MMP23B,NaN,NaN,5,3.164,NaN,rs115866895,1
2,ENSG00000248333,CDK11B,1,1570603,1590473,-1,protein_coding,728642.0,CDK11B,7.282944e-01,NaN,34,11.560,1.617240e-15,rs115866895,1
3,ENSG00000189339,SLC35E2B,1,1592939,1624167,-1,protein_coding,728661.0,SLC35E2B,6.210025e-01,-0.794611,34,11.560,1.617240e-15,rs115866895,1
4,ENSG00000157873,TNFRSF14,1,2487078,2496821,1,protein_coding,8764.0,TNFRSF14,8.199991e-01,-0.131268,1,1.637,NaN,rs4648450,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
592,ENSG00000196419,XRCC6,22,42017123,42060044,1,protein_coding,2547.0,XRCC6,9.968559e-01,-0.128101,2,6.981,NaN,rs28489620,314
593,ENSG00000100138,NHP2L1,22,42069934,42086508,-1,protein_coding,4809.0,NHP2L1,7.105853e-01,0.082226,2,6.981,NaN,rs28489620,314
594,ENSG00000167077,MEI1,22,42095503,42195460,1,protein_coding,150365.0,MEI1,1.215924e-08,0.500155,13,8.496,NaN,rs28489620,314
595,ENSG00000100147,CCDC134,22,42196683,42222303,1,protein_coding,79879.0,CCDC134,7.411728e-04,0.324960,14,8.496,NaN,rs28489620,314


### Construct the PCAs in terms of Genes

In [253]:
gene_pca_dict = {} #pd.DataFrame(index = genes_df.symbol.values)
# Get the possible PCAs
pc_gene_ours_df = genes_df.loc[:,["IndSigSNPs","symbol"]]
pc_gene_ours_with_exp_df = genes_df.loc[:,["IndSigSNPs","symbol"]]
pc_gene_pap_df = genes_df.loc[:,["IndSigSNPs","symbol"]]
pc_gene_dict = {"pap": pc_gene_pap_df,
                "ours": pc_gene_ours_df,
                "ours_with_exp": pc_gene_ours_with_exp_df}
pc_dict = {"pap": pc_pap_df,
            "ours": pc_ours_df,
            "ours_with_exp": pc_ours_with_exp_df}

In [254]:
for lab in pc_gene_dict.keys():
   pc_gene_df = pc_gene_dict[lab]
   pc_df = pc_dict[lab]
   pc_gene_df[pc_df.columns] = 0.0
   for snp in pc_gene_df.IndSigSNPs:
      if ";" in snp:
         snp_list = list(snp.split(";"))
      else: snp_list = [snp]
      for pc_col in pc_df.columns:
         for snp in snp_list:
            gene_rows = list(pc_gene_df[pc_gene_df["IndSigSNPs"]==snp].index)
            pc_gene_df.loc[gene_rows, pc_col] = pc_df.loc[snp, pc_col]
pc_gene_df

,IndSigSNPs,symbol,pc_1,pc_2
0,rs115866895,MIB2,-6.436117,0.075144
1,rs115866895,MMP23B,-6.436117,0.075144
2,rs115866895,CDK11B,-6.436117,0.075144
3,rs115866895,SLC35E2B,-6.436117,0.075144
4,rs4648450,TNFRSF14,-10.560462,-1.307634
...,...,...,...,...
592,rs28489620,XRCC6,-6.469766,7.800513
593,rs28489620,NHP2L1,-6.469766,7.800513
594,rs28489620,MEI1,-6.469766,7.800513
595,rs28489620,CCDC134,-6.469766,7.800513


### Save the gene list

In [255]:
pc_gene_df.symbol.to_csv(path_dir+"/gene_list.csv", index = False, header=False)

#### Load the pathways found on reactome

In [256]:
reactome_pathways = pd.read_csv(path_dir+"/reactome_pathways.csv")
reactome_pathways["Submitted entities found"] = reactome_pathways["Submitted entities found"].str.split(";")
#reactome_pathways = reactome_pathways.explode("Submitted entities found")
len(reactome_pathways["Pathway name"].unique())

1346

### Group the genes by association
In order to get pathway enrichment we need to be able to group the genes. For each principal componene group the genes by their association score.

In [257]:
for lab in pc_gene_dict.keys():
   pc_gene_df = pc_gene_dict[lab]
   pc_df = pc_dict[lab]
   for pc_col in pc_df.columns:
     groups = pd.cut(pc_gene_df[pc_col],
                     bins = 10,
                     labels = [pc_col+"_%d"%i for i in range(10)])
     groups_ranges = pd.cut(pc_gene_df[pc_col],
                     bins = 10)
     pc_gene_df[pc_col+"_bin"] = groups
     pc_gene_df[pc_col+"_bin_ranges"] = groups_ranges


### Restructure the dataframe to have the bin labels as columns and genes as the values.

In [258]:
pc_gene_groups_dict = {}
bin_range_dict = {}
for lab in pc_gene_dict.keys():
   pc_gene_df = pc_gene_dict[lab]
   bin_groups = {}
   bin_ranges = {}
   for pc_col in pc_df.columns:
      for bin in pc_gene_df[pc_col +"_bin"].unique():
         genes = pc_gene_df.loc[pc_gene_df[pc_col+"_bin"]==bin,"symbol"]
         bin_groups[bin] = genes
         bin_ranges[bin] = {"mid": 
                            pc_gene_df.loc[pc_gene_df[pc_col+"_bin"]==bin,pc_col+"_bin_ranges"].values[0].mid,
                            "interval": pc_gene_df.loc[pc_gene_df[pc_col+"_bin"]==bin,pc_col+"_bin_ranges"].values[0]
         }
   pc_gene_groups_dict[lab] = pd.DataFrame(bin_groups)
   bin_range_dict[lab] = pd.DataFrame(bin_ranges).T
   #print(pc_gene_groups_dict[lab])
bin_range_dict

{'pap':            mid           interval
 pc_1_4 -2.3055    (-3.271, -1.34]
 pc_1_6  1.5555      (0.59, 2.521]
 pc_1_3  -4.236   (-5.201, -3.271]
 pc_1_7   3.486     (2.521, 4.451]
 pc_1_0 -10.037  (-11.012, -9.062]
 pc_1_8  5.4165     (4.451, 6.382]
 pc_1_5  -0.375      (-1.34, 0.59]
 pc_1_9   7.347     (6.382, 8.312]
 pc_1_2 -6.1665   (-7.132, -5.201]
 pc_1_1  -8.097   (-9.062, -7.132]
 pc_2_6  3.4175     (2.357, 4.478]
 pc_2_2 -5.0655   (-6.126, -4.005]
 pc_2_5  1.2965     (0.236, 2.357]
 pc_2_3  -2.945   (-4.005, -1.885]
 pc_2_4 -0.8245    (-1.885, 0.236]
 pc_2_1 -7.1865   (-8.247, -6.126]
 pc_2_7   5.538     (4.478, 6.598]
 pc_2_8  7.6585     (6.598, 8.719]
 pc_2_0 -9.3175  (-10.388, -8.247]
 pc_2_9  9.7795     (8.719, 10.84],
 'ours':             mid            interval
 pc_1_7    7.346      (5.881, 8.811]
 pc_1_8  10.2765     (8.811, 11.742]
 pc_1_2  -7.3075    (-8.773, -5.842]
 pc_1_4 -1.44645    (-2.912, 0.0191]
 pc_1_9  13.2075    (11.742, 14.673]
 pc_1_0  -13.184  (-14.664,

## ENRICHR
### Get the pathway association for gene sets in each cluster.
Using the ENRICHR api get the pathways associated with the sets of SNPS in each cluster.

In [259]:
ENRICHR_add_URL = 'https://maayanlab.cloud/Enrichr/addList'
ENRICHR_URL = 'https://maayanlab.cloud/Enrichr/enrich'
gene_set_library = 'Enrichr_Users_Contributed_Lists_2020'
query_string = '?userListId=%s&backgroundType=%s'
path_clust_OR_dict ={}
path_clust_score_dict ={}
s= requests.Session()
retry = Retry(connect=3, backoff_factor=0.5)
adapter = HTTPAdapter(max_retries=retry)
s.mount('http://', adapter)
s.mount('https://', adapter)
user_ids_full_list = {}
for c_lab in cluster_method_labels:
    print(c_lab)
    clust_genes_df = gene_clust_dict[c_lab]
    c_num_labs = clust_genes_df.columns
    OR_list = []
    score_list = []
    user_ids_list = []
    all_list = []
    for c_num_lab in c_num_labs:
        try: c_num = int(c_num_lab[-1])
        except: break
        clust_cnum_genes = clust_genes_df.index[clust_genes_df[c_num_lab] == 1].tolist()
        genes_str = '\n'.join(clust_cnum_genes)
        description = c_lab + ' gene list'
        payload = {
            'list': (None, genes_str),
            'description': (None, description)
        }
        response = s.post(ENRICHR_add_URL, files=payload)
        if not response.ok:
            #raise Exception
            print('Error analyzing gene list')
        else:
            data = json.loads(response.text)
            #print(data)
            user_list_id = data['userListId']
            #print(user_list_id)
            user_ids_list.append(user_list_id)
            response = s.get(
            ENRICHR_URL + query_string % (user_list_id, gene_set_library)
            )
            if not response.ok:
                #raise Exception
                print('Error fetching enrichment results')
            else:
                data = json.loads(response.text)[gene_set_library]
                if len(data)>0:
                    for i in range(len(data)):
                        rank = data[i][0]
                        pathway = data[i][1]
                        pval = data[i][2]
                        OR = data[i][3]
                        score = data[i][4]
                        overlap_genes = data[i][5]
                        adjust_pval = data[i][6]
                        all_row = {"cluster_number": c_num,
                            "pathway": pathway,
                            "rank": rank,
                            "pval": pval, 
                            "adjusted_pval": adjust_pval,
                            "Odds_Ratio" : OR,
                            "combined_score": score,
                            "n_overlap": len(overlap_genes),
                            "overlap_genes": overlap_genes}
                        all_list.append(all_row)
                        OR_row = {"cluster_number": c_num,
                            "pathway": pathway,
                            "Odds_Ratio" : OR}
                        OR_list.append(OR_row)
                        score_row = {"cluster_number": c_num,
                                "pathway": pathway,
                                "combined_score": score}
                        score_list.append(score_row)
                else:
                    OR_row = {"cluster_number": c_num,
                              "pathway": "No_pathway"}
                    OR_list.append(OR_row)
                    score_row = {"cluster_number": c_num,
                              "pathway": "No_pathway"}
                    score_list.append(score_row)
    user_ids_full_list[c_lab] = user_ids_list
    OR_df = pd.DataFrame(OR_list)
    score_df = pd.DataFrame(score_list)
    all_df = pd.DataFrame(all_list)
    OR_df.to_csv(path_dir+"/clust_paths_OR_"+c_lab+".csv")
    score_df.to_csv(path_dir+"/clust_path_combined_score_"+c_lab+".csv")
    all_df.to_csv(path_dir+"/clust_path_enrichment_"+c_lab+".csv")
    path_clust_OR_dict[c_lab] = OR_df
    path_clust_score_dict[c_lab] = score_df
print(path_clust_score_dict)
    

KmeansLloydStdFourDarrous
KmeansLloydStdSixDarrous
KmeansElkanStdFourDarrous
KmeansLloydStdFourProposed
KmeansLloydStdSixProposed
KmeansElkanStdFourProposed
KmedoidsEuclideanProposedFour
KmeansCosineSixProposed
DBScanZeroProposed
DBScanOneProposed
DBScanTwoProposed
KmeansCosineMinibatchSixProposed
KmedoidsCosineSixProposed
GMMZeroProposed
GMMOneProposed
BirchZeroProposed
Error analyzing gene list
Error analyzing gene list
Error analyzing gene list
Error analyzing gene list
Error analyzing gene list
Error analyzing gene list
Error analyzing gene list
Error analyzing gene list
Error analyzing gene list
Error analyzing gene list
Error analyzing gene list
Error analyzing gene list
Error analyzing gene list
Error analyzing gene list
Error analyzing gene list
Error analyzing gene list
Error analyzing gene list
Error analyzing gene list
Error analyzing gene list
Error analyzing gene list
Error analyzing gene list
Error analyzing gene list
Error analyzing gene list
Error analyzing gene list
Er

### Get the pathway association for gene sets in each PCA bin

In [260]:
gene_set_library = 'Enrichr_Users_Contributed_Lists_2020'
query_string = '?userListId=%s&backgroundType=%s'
path_pca_OR_dict ={}
path_pca_score_dict ={}
path_pca_adj_score_dict = {}
s= requests.Session()
retry = Retry(connect=3, backoff_factor=0.5)
adapter = HTTPAdapter(max_retries=retry)
s.mount('http://', adapter)
s.mount('https://', adapter)
user_ids_full_list = {}
for lab in pc_gene_groups_dict.keys():
    pc_genes_df = pc_gene_groups_dict[lab]
    bin_labs = pc_genes_df.columns
    OR_list = []
    score_list = []
    adj_score_list = []
    user_ids_list = []
    all_list = []
    for bin in bin_labs:
        bin_genes = pc_genes_df[bin].dropna(axis=0).tolist()
        bin_mid = bin_range_dict[lab].loc[bin,"mid"]
        genes_str = '\n'.join(bin_genes)
        description = bin + ' gene list'
        payload = {
            'list': (None, genes_str),
            'description': (None, description)
        }
        response = s.post(ENRICHR_add_URL, files=payload)
        if not response.ok:
            #raise Exception
            print('Error analyzing gene list')
        else:
            data = json.loads(response.text)
            #print(data)
            user_list_id = data['userListId']
            #print(user_list_id)
            user_ids_list.append(user_list_id)
            response = s.get(
            ENRICHR_URL + query_string % (user_list_id, gene_set_library)
            )
            if not response.ok:
                #raise Exception
                print('Error fetching enrichment results')
            else:
                data = json.loads(response.text)[gene_set_library]
                if len(data)>0:
                    for i in range(len(data)):
                        rank = data[i][0]
                        pathway = data[i][1]
                        pval = data[i][2]
                        OR = data[i][3]
                        score = data[i][4]
                        adj_score = score*bin_mid
                        overlap_genes = data[i][5]
                        adjust_pval = data[i][6]
                        all_row = {"bin": bin,
                            "pathway": pathway,
                            "rank": rank,
                            "pval": pval, 
                            "adjusted_pval": adjust_pval,
                            "Odds_Ratio" : OR,
                            "combined_score": score,
                            "adjusted_score": adj_score,
                            "n_overlap": len(overlap_genes),
                            "overlap_genes": overlap_genes}
                        all_list.append(all_row)
                        OR_row = {"bin": bin,
                            "pathway": pathway,
                            "Odds_Ratio" : OR}
                        OR_list.append(OR_row)
                        score_row = {"bin": bin,
                                "pathway": pathway,
                                "combined_score": score}
                        score_list.append(score_row)
                        adj_score_row = {"bin": bin,
                                "pathway": pathway,
                                "adjusted_score": adj_score}
                        adj_score_list.append(adj_score_row)
                else:
                    OR_row = {"bin": bin,
                              "pathway": "No_pathway"}
                    OR_list.append(OR_row)
                    score_row = {"bin": bin,
                              "pathway": "No_pathway"}
                    score_list.append(score_row)
                    adj_score_row = {"bin": bin,
                              "pathway": "No_pathway"}
                    adj_score_list.append(adj_score_row)
    user_ids_full_list[bin] = user_ids_list
    OR_df = pd.DataFrame(OR_list)
    score_df = pd.DataFrame(score_list)
    all_df = pd.DataFrame(all_list)
    adj_score_df = pd.DataFrame(adj_score_list)
    OR_df.to_csv(path_dir+"/pca_paths_OR_"+lab+".csv")
    score_df.to_csv(path_dir+"/pca_path_combined_score_"+lab+".csv")
    all_df.to_csv(path_dir+"/pca_path_enrichment_"+lab+".csv")
    adj_score_df.to_csv(path_dir + "/pca_path_adj_score"+lab+".csv")
    path_pca_OR_dict[lab] = OR_df
    path_pca_score_dict[lab] = score_df
    path_pca_adj_score_dict[lab] = adj_score_df
    

In [261]:
# OFFLINE VERSION - load latest 
path_clust_OR_dict = {}
path_clust_score_dict = {}
user_ids_full_list = {}
for c_lab in cluster_method_labels:
    clust_genes_df = gene_clust_dict[c_lab]
    OR_df = pd.read_csv(path_dir+"/clust_paths_OR_"+c_lab+".csv", index_col=0)
    score_df = pd.read_csv(path_dir+"/clust_path_combined_score_"+c_lab+".csv", index_col=0)
    all_df = pd.read_csv(path_dir+"/clust_path_enrichment_"+c_lab+".csv", index_col=0)
    path_clust_OR_dict[c_lab] = OR_df
    path_clust_score_dict[c_lab] = score_df

# Filter the dominant pathways for each cluster.
Get the top fifty pathways overall for each cluster method by `combined score` and `odds_ratio`.
Also get the top 5 pathways per cluster by `combined_score` and `odds_ratio`.

The odds ratio is computed using this formula:

$$ oddsRatio = \frac{1.0 * a * d}{max(1.0 * b * c, 1)}$$ 

where: a are the overlapping genes, b are the genes in the annotated set - overlapping genes, c- are the genes in the input set - overlapping genes, and d- are the 20,000 genes (or total genes in the background) - genes in the annotated set - genes in the input set + overlapping genes

Finally, the combined score is a combination of the p-value and odds ratio calculated by multiplying the two scores as follows:

$$ c = -log(p) * \text{oddsRatio} $$
Where c is the combined score, p is the p-value computed using Fisher's exact test, and oddsRatio is the odds ratio. The combined score provides a compromise between both methods and in several benchmarks we show that it reports the best rankings when compared with the other scoring schemes.

Enrichr provides all four options for sorting enriched terms.

In [262]:
c_lab = cluster_method_labels[0]
print(path_clust_OR_dict[c_lab])

      cluster_number                                            pathway  \
0                  1  lead SNPs from Neale v2 analysis of UKBB data,...   
1                  1  https://science.sciencemag.org/content/365/645...   
2                  1  10 percent of the longest transcripts of the h...   
3                  1                                             ct1346   
4                  1                                       inflammation   
...              ...                                                ...   
3676               3      Blue module gene list CRC (Normal and Cancer)   
3677               3  Bacteria translocation, dataset: 4b799ecdbaaed...   
3678               3  Skin Cutaneous Melanoma under expressed genes ...   
3679               3                    MEF-iPS diff ES DNA methylation   
3680               3                                     pred all genes   

      Odds_Ratio  
0      21.038178  
1       4.110373  
2       3.457802  
3       2.345616  
4   

In [263]:
path_tops_dict = {}
top_five_lab = "TopFive"
top_fifty_lab = "TopFifty"
score_lab = "CombinedScore"
OR_lab = "OddsRatio"
for lab in cluster_method_labels:
    print(lab)
    print(path_clust_OR_dict[lab])
    sorted_df = path_clust_OR_dict[lab].sort_values(by=OR_lab, ascending=False)
    df_wide = sorted_df.pivot_table(index='pathway', columns='cluster_number', values=OR_lab)
    ind_list = df_wide.index
    col_list = df_wide.columns
    df_wide = pd.DataFrame(data = np.nan_to_num(df_wide),
                           index=ind_list,
                           columns = col_list)
    df_long= pd.melt(df_wide, ignore_index=False, value_name=OR_lab)
    df_long= df_long.reset_index().sort_values(by=OR_lab, ascending=False)
    top_fifty_byOR_df = df_long.head(50)
    top_five_perclust_df = df_long.groupby("cluster_number").head(5)
    OR_dict = {top_five_lab:top_five_perclust_df, top_fifty_lab:top_fifty_byOR_df}
    sorted_df = path_clust_score_dict[lab].sort_values(by=score_lab, ascending=False)
    df_wide = sorted_df.pivot_table(index='pathway', columns='cluster_number', values=score_lab)
    ind_list = df_wide.index
    col_list = df_wide.columns
    df_wide = pd.DataFrame(data = np.nan_to_num(df_wide),
                           index=ind_list,
                           columns = col_list)
    df_long= pd.melt(df_wide, ignore_index=False, value_name=score_lab)
    df_long= df_long.reset_index().sort_values(by=score_lab, ascending=False)
    top_fifty_byscore_df = df_long.head(50)
    top_five_perclust_df = df_long.groupby("cluster_number").head(5)
    score_dict = {top_five_lab:top_five_perclust_df, top_fifty_lab:top_fifty_byscore_df}
    path_tops_dict[lab] = {OR_lab:OR_dict, score_lab:score_dict}



KmeansLloydStdFourDarrous
      cluster_number                                            pathway  \
0                  1  lead SNPs from Neale v2 analysis of UKBB data,...   
1                  1  https://science.sciencemag.org/content/365/645...   
2                  1  10 percent of the longest transcripts of the h...   
3                  1                                             ct1346   
4                  1                                       inflammation   
...              ...                                                ...   
3676               3      Blue module gene list CRC (Normal and Cancer)   
3677               3  Bacteria translocation, dataset: 4b799ecdbaaed...   
3678               3  Skin Cutaneous Melanoma under expressed genes ...   
3679               3                    MEF-iPS diff ES DNA methylation   
3680               3                                     pred all genes   

      Odds_Ratio  
0      21.038178  
1       4.110373  
2       3.457802

In [264]:
path_pca_tops_dict = {}
adj_score_lab = "adjusted_score"
for lab in pc_gene_groups_dict.keys():
    sorted_df = path_pca_OR_dict[lab].sort_values(by=OR_lab, ascending=False)
    df_wide = sorted_df.pivot_table(index='pathway', columns='bin', values=OR_lab)
    ind_list = df_wide.index
    col_list = df_wide.columns
    df_wide = pd.DataFrame(data = np.nan_to_num(df_wide),
                           index=ind_list,
                           columns = col_list)
    df_long= pd.melt(df_wide, ignore_index=False, value_name=OR_lab)
    df_long= df_long.reset_index()
    df_long_sorted = df_long.sort_values(by=OR_lab, ascending = False)
    top_fifty_byOR_df = df_long_sorted.head(50)
    top_five_perclust_df = df_long_sorted.groupby("bin").head(5)
    OR_dict = {top_five_lab:top_five_perclust_df, top_fifty_lab:top_fifty_byOR_df}
    sorted_df = path_pca_score_dict[lab].sort_values(by=score_lab, ascending=False)
    df_wide = sorted_df.pivot_table(index='pathway', columns='bin', values=score_lab)
    ind_list = df_wide.index
    col_list = df_wide.columns
    df_wide = pd.DataFrame(data = np.nan_to_num(df_wide),
                           index=ind_list,
                           columns = col_list)
    df_long= pd.melt(df_wide, ignore_index=False, value_name=score_lab)
    df_long= df_long.reset_index()
    df_long_sorted = df_long.sort_values(by=score_lab, ascending = False)
    top_fifty_byscore_df = df_long_sorted.head(50)
    top_five_perclust_df = df_long_sorted.groupby("bin").head(5)
    score_dict = {top_five_lab:top_five_perclust_df, top_fifty_lab:top_fifty_byscore_df}
    sorted_df = path_pca_adj_score_dict[lab].sort_values(by=adj_score_lab, ascending=False)
    df_wide = sorted_df.pivot_table(index='pathway', columns='bin', values=adj_score_lab)
    ind_list = df_wide.index
    col_list = df_wide.columns
    df_wide = pd.DataFrame(data = np.nan_to_num(df_wide),
                           index=ind_list,
                           columns = col_list)
    df_long= pd.melt(df_wide, ignore_index=False, value_name=adj_score_lab)
    df_long= df_long.reset_index()
    df_long_sorted = df_long.sort_values(by=adj_score_lab, ascending = False)
    top_fifty_byscore_df = df_long_sorted.head(50)
    top_five_perclust_df = df_long_sorted.groupby("bin").head(5)
    adj_score_dict = {top_five_lab:top_five_perclust_df, top_fifty_lab:top_fifty_byscore_df}
    path_pca_tops_dict[lab] = {OR_lab:OR_dict, score_lab:score_dict, adj_score_lab: adj_score_dict}
path_pca_tops_dict



{'pap': {'Odds_Ratio': {'top_five':                                                  pathway     bin    Odds_Ratio
   13762                                wild and pearl mice  pc_2_0  31570.000000
   13763                                       wild vs pale  pc_2_0  31546.000000
   23909                            Basal 2 control Vs COPD  pc_2_9    203.551020
   2672                                         CancerGenes  pc_1_2    201.808081
   22864  Genes associated with osteonecrosis in ALL chi...  pc_2_8    159.752000
   ...                                                  ...     ...           ...
   18821  557gene litte list, dataset: c31f4fef2910f4f50...  pc_2_5     12.088415
   19205  Inflammatory genes with most significant isofo...  pc_2_5     11.050195
   20025                                            wodecuo  pc_2_5     10.988914
   18155                  Ribavirin-Down Sars-Cov-2-S7-1-Up  pc_2_4     10.802832
   17704             CircRNA in mouse developmental tissues  pc_2

In [265]:
top_fifty_byscore_df

,pathway,bin,adjusted_score
11063,"lead SNPs from Neale v2 analysis of UKBB data,...",pc_1_8,29196.217453
22331,"lead SNPs from Neale v2 analysis of UKBB data,...",pc_2_7,7185.850801
9811,"lead SNPs from Neale v2 analysis of UKBB data,...",pc_1_7,6120.227832
21079,"lead SNPs from Neale v2 analysis of UKBB data,...",pc_2_6,6021.470935
12315,"lead SNPs from Neale v2 analysis of UKBB data,...",pc_1_9,4750.949892
23909,Basal 2 control Vs COPD,pc_2_9,3273.073021
23195,SLE risk genes,pc_2_8,2880.320981
23154,Rat proximal tubule TF network identified by wTO.,pc_2_8,2766.612886
24158,HMGB1-interacted proteins pathway analysis,pc_2_9,2599.979875
11331,ALX1 target genes,pc_1_9,2430.301930


In [266]:
top_five_perclust_df

,pathway,bin,adjusted_score
11063,"lead SNPs from Neale v2 analysis of UKBB data,...",pc_1_8,29196.217453
22331,"lead SNPs from Neale v2 analysis of UKBB data,...",pc_2_7,7185.850801
9811,"lead SNPs from Neale v2 analysis of UKBB data,...",pc_1_7,6120.227832
21079,"lead SNPs from Neale v2 analysis of UKBB data,...",pc_2_6,6021.470935
12315,"lead SNPs from Neale v2 analysis of UKBB data,...",pc_1_9,4750.949892
...,...,...,...
6053,laminB1 over3,pc_1_4,0.000000
382,"Hashimoto, 1.2 fold up",pc_1_0,0.000000
381,Harvard home work,pc_1_0,0.000000
380,HYDROXYCHLOROQUINE-UP SARS-COV-2-UP-Cluster-1,pc_1_0,0.000000


## Plotting functions

### Heatmap plots for comparing cluster pathway association.

In [267]:
def chart_cluster_pathway(data_array, x_lab, y_lab, z_lab, title_str, text_precision = ".0f"):
    # Convert this grid to columnar data expected by Altair
    title = alt.TitleParams(title_str, anchor='middle')
    chart = alt.Chart(data_array, title = title).mark_rect().encode(
        alt.X(x_lab+":O").title(x_lab),
        alt.Y(y_lab+":O").title(y_lab),
        alt.Color(z_lab+":Q").title(z_lab)
        ).properties(
            width=400,
            height=400
        )
    text = chart.mark_text(baseline='middle').encode(
    alt.Text(z_lab+':Q', format = text_precision),
    color=alt.value('black')
)
    return(chart+text)

### Plot the top five for each cluster and the top fifty pathways overall. For Odds-ratio and combined-score

In [268]:
x_lab = "cluster_number"
y_lab = "pathway"
tex_list = []
for lab in cluster_method_labels:
    lab = lab.replace("_"," ").title().replace(" ","")
    title_str_per = '{} pathways per cluster for cluster method {}'
    title_str_ove = '{} pathways over all clusters for cluster method {}'
    save_str = path_plots+ "/{}ClusterPaths{}{}.png"
    # OR plots 
    # - Top 5 per clust
    top_five_or_title = title_str_per.format(top_five_lab,lab)
    df = path_tops_dict[lab][OR_lab][top_five_lab]
    chart = chart_cluster_pathway(df,x_lab, y_lab, OR_lab, top_five_or_title)
    fname = save_str.format(top_five_lab, OR_lab,lab)
    chart.save(fname)
    tex_list +=[pathway_figure_tex(lab,  fname= fname, score = "Odds Ratio", filter = "top five per cluster")]
    # - Top 50
    top_fifty_or_title = title_str_ove.format(top_fifty_lab, lab)
    chart = chart_cluster_pathway(path_tops_dict[lab][OR_lab][top_fifty_lab],x_lab, y_lab, OR_lab, top_fifty_or_title)
    fname = save_str.format(top_fifty_lab, OR_lab, lab)
    chart.save(fname)
    tex_list +=[pathway_figure_tex(lab, fname= fname, score = "Odds Ratio", filter = "top fifty overall")]
    # Combined score
    # - Top 5 per clust
    top_five_score_title = title_str_per.format(top_five_lab, lab)
    chart = chart_cluster_pathway(path_tops_dict[lab][score_lab][top_five_lab],x_lab, y_lab, score_lab, top_five_score_title)
    fname = save_str.format(top_five_lab, score_lab,lab)
    chart.save(fname)
    tex_list +=[pathway_figure_tex(lab,  fname= fname, score = "Combined Score", filter = "top five per cluster")]
    # - Top 50
    top_fifty_score_title = title_str_ove.format(top_fifty_lab, lab)
    chart = chart_cluster_pathway(path_tops_dict[lab][score_lab][top_fifty_lab],x_lab, y_lab, score_lab, top_fifty_score_title)
    fname= save_str.format(top_fifty_lab, score_lab,lab)
    chart.save(fname)
    tex_list +=[pathway_figure_tex(lab, fname= fname, score = "Combined Score", filter = "top fifty overall")]
chart


alt.LayerChart(...)

In [269]:
x_lab = "bin"
y_lab = "pathway"
for lab in path_pca_tops_dict.keys():
    title_str = '{} pathways per PCA bin for transform {}'
    save_str = path_plots+ "/{}_pca_paths_{}_{}.png"
    # OR plots 
    # - Top 5 per clust
    top_five_or_title = title_str.format(top_five_lab,lab)
    df = path_pca_tops_dict[lab][OR_lab][top_five_lab]
    chart = chart_cluster_pathway(df,x_lab, y_lab, OR_lab, top_five_or_title)
    fname = save_str.format(top_five_lab, OR_lab, lab)
    chart.save(fname)
    # # - Top 50
    top_fifty_or_title = title_str.format(top_fifty_lab, lab)
    chart = chart_cluster_pathway(path_pca_tops_dict[lab][OR_lab][top_fifty_lab],x_lab, y_lab, OR_lab, top_fifty_or_title)
    fname = save_str.format(top_fifty_lab, OR_lab, lab)
    chart.save(fname)
    # Combined score
    # - Top 5 per clust
    top_five_score_title = title_str.format(top_five_lab, lab)
    chart = chart_cluster_pathway(path_pca_tops_dict[lab][score_lab][top_five_lab],x_lab, y_lab, score_lab, top_five_score_title)
    fname = save_str.format(top_five_lab, score_lab,lab)
    chart.save(fname)
    # - Top 50
    top_fifty_score_title = title_str.format(top_fifty_lab, lab)
    chart = chart_cluster_pathway(path_pca_tops_dict[lab][score_lab][top_fifty_lab],x_lab, y_lab, score_lab, top_fifty_score_title)
    fname = save_str.format(top_fifty_lab, score_lab,lab)
    chart.save(fname)
    # Adjusted Combined score
    # - Top 5 per clust
    top_five_score_title = title_str.format(top_five_lab, lab)
    chart = chart_cluster_pathway(path_pca_tops_dict[lab][adj_score_lab][top_five_lab],x_lab, y_lab, adj_score_lab, top_five_score_title)
    fname = save_str.format(top_five_lab, adj_score_lab,lab)
    chart.save(fname)
    # - Top 50
    top_fifty_score_title = title_str.format(top_fifty_lab, lab)
    chart = chart_cluster_pathway(path_pca_tops_dict[lab][adj_score_lab][top_fifty_lab],x_lab, y_lab, adj_score_lab, top_fifty_score_title)
    fname = save_str.format(top_fifty_lab, adj_score_lab,lab)
    chart.save(fname)
chart


alt.LayerChart(...)

# Comparing the cluster methods
In an ideal scenario our clustering method would:
* identify pathways which have not been split accross other clusters. 
* separate the pathways from each other so they can be identified.

In addition to this we also want to consider whether the pathways identified have significant enrichment and if so utilise this to inform further treatment investigation.

## How well has each cluster methods performed on not splitting clusters up?

In [270]:
def ssd(A,B):
  dif = A.ravel() - B.ravel()
  return abs(dif).sum()/(len(A.ravel()))
def uniqueness(df, axis = 0, score_lab = "combined_score"):
    df_wide = df.pivot_table(index='pathway', columns='cluster_number', values=score_lab)
    if df_wide.shape[1]==1 and axis ==1: return("NaN")
    mat = np.nan_to_num(df_wide.to_numpy())
    if axis == 1:
      mat = mat.T
    mat_norm = (mat - mat.min(0))/ mat.ptp(0)
    max_mat = np.zeros(mat_norm.shape)
    max_mat[np.argmax(mat_norm, axis = 0)] = 1
    score = ssd(max_mat, mat_norm)
    return(score)
def clust_path_score(df, score_lab = score_lab):
  path_contain_score = uniqueness(df, axis = 0, score_lab=score_lab)
  path_separate_score = uniqueness(df, axis = 1, score_lab=score_lab)
  out_dict = {"PathContaining": path_contain_score,
               "PathSeparating": path_separate_score}
  return(out_dict)

In [271]:
def rescale_col (df, col_lab):
  num_col = pd.to_numeric(df[col_lab], errors='coerce')
  not_nan_rows = num_col.notnull()
  num_col[not_nan_rows] = (num_col[not_nan_rows] - num_col[not_nan_rows].min())/ (num_col[not_nan_rows].max()- num_col[not_nan_rows].min())
  return(num_col[not_nan_rows])

In [272]:
clust_method_scores_df.PathContaining

8     0.000000
17    0.168617
16    0.168617
18    0.168617
19    0.192361
14    0.081227
13    0.263454
11    0.189508
7     0.202920
12    0.106824
6     0.189003
5     0.192186
3     0.211330
2     0.249573
0     0.249573
4     0.179412
1     0.255066
15    1.000000
9     0.081976
10    0.081976
Name: PathContaining, dtype: float64

In [273]:
clust_method_scores = []
for lab in cluster_method_labels:
    clust_path_dict = clust_path_score(path_clust_OR_dict[lab], score_lab=OR_lab)
    clust_path_dict["ClusterMethod"]=  lab
    clust_method_scores.append(clust_path_dict)
# Rescale the cluster scores.
cont_weight = 0.90
sep_weight = 0.1
clust_method_scores_df = pd.DataFrame(clust_method_scores)
clust_method_scores_df.PathContaining = rescale_col(clust_method_scores_df, "PathContaining")
clust_method_scores_df.PathSeparating = rescale_col(clust_method_scores_df, "PathSeparating")
clust_method_scores_df["OverallPathwayIdentifying"] = cont_weight * clust_method_scores_df.PathContaining + sep_weight * clust_method_scores_df.PathSeparating
clust_method_scores_df.sort_values(by="OverallPathwayIdentifying", ascending=True, na_position="last")

,PathContaining,PathSeparating,ClusterMethod,OverallPathwayIdentifying
8,0.000000,0.000000,DBScanZeroProposed,0.000000
14,0.081391,0.452547,GMMOneProposed,0.118507
17,0.168957,0.000000,BirchTwoProposed,0.152061
16,0.168957,0.000000,BirchOneProposed,0.152061
18,0.168957,0.000000,KmeansCosineTwoMinAICProposed,0.152061
12,0.107039,0.602011,KmedoidsCosineSixProposed,0.156536
19,0.192749,0.159654,KmedoidsCosineThreeMinAICProposed,0.189440
6,0.189384,0.250035,KmedoidsEuclideanProposedFour,0.195449
5,0.192573,0.230169,KmeansElkanStdFourProposed,0.196333
3,0.192573,0.230169,KmeansLloydStdFourProposed,0.196333


## Account for overfitting
Divide the cluster pathway score by the `AIC`. This will adjust for methods which are overfitting clusters.

In [274]:
clust_method_scores_df["aic"] = [clustering_info.loc[lab, "aic"] for lab in clust_method_scores_df.ClusterMethod]
clust_method_scores_df["AdjustedPathContaining"] = clust_method_scores_df.PathContaining*clust_method_scores_df.aic
clust_method_scores_df["AdjustedPathSeparating"] = clust_method_scores_df.PathSeparating*clust_method_scores_df.aic
clust_method_scores_df.AdjustedPathContaining = rescale_col(clust_method_scores_df, "AdjustedPathContaining")
clust_method_scores_df.AdjustedPathSeparating = rescale_col(clust_method_scores_df, "AdjustedPathSeparating")
clust_method_scores_df["AdjustedOverall"] = cont_weight*clust_method_scores_df.AdjustedPathContaining + sep_weight*clust_method_scores_df.AdjustedPathSeparating

clust_method_scores_df.sort_values(by="AdjustedOverall", ascending=True, na_position="last",inplace=True)
clust_method_scores_df.to_csv(tex_dir+"/ClusterPathwayComparions.csv", 
                              float_format='%.3f')
clust_method_scores_df

,PathContaining,PathSeparating,ClusterMethod,OverallPathwayIdentifying,aic,AdjustedPathContaining,AdjustedPathSeparating,AdjustedOverall
8,0.000000,0.000000,DBScanZeroProposed,0.000000,5698.360008,0.000000,0.000000,0.000000
17,0.168957,0.000000,BirchTwoProposed,0.152061,1959.091867,0.001250,0.000000,0.001125
16,0.168957,0.000000,BirchOneProposed,0.152061,1959.091867,0.001250,0.000000,0.001125
18,0.168957,0.000000,KmeansCosineTwoMinAICProposed,0.152061,2953.690816,0.001885,0.000000,0.001697
19,0.192749,0.159654,KmedoidsCosineThreeMinAICProposed,0.189440,6053.822311,0.004408,0.003651,0.004332
14,0.081391,0.452547,GMMOneProposed,0.118507,9840.977985,0.003026,0.016824,0.004406
13,0.263985,0.115324,GMMZeroProposed,0.249119,5537.238223,0.005522,0.002412,0.005211
11,0.189890,0.513248,KmeansCosineMinibatchSixProposed,0.222226,6597.522864,0.004733,0.012792,0.005539
7,0.203330,0.457834,KmeansCosineSixProposed,0.228780,7170.450067,0.005508,0.012402,0.006197
12,0.107039,0.602011,KmedoidsCosineSixProposed,0.156536,10553.675863,0.004268,0.024002,0.006241


In [276]:
with open(tex_dir+"/cluster_pathway_figures_text.tex", "w") as text_file:
    for tex_item in tex_list:
        text_file.write(tex_item)